In [1]:
import pandas as pd
import json

In [2]:
import redis

redis_client = redis.Redis(host='localhost', port=6399, db=0,decode_responses=True)

In [3]:
from qdrant_client import QdrantClient
from qdrant_client.models import  Distance, VectorParams

client = QdrantClient(url="http://localhost:6333")

from qdrant_client.models import PointStruct
from uuid import uuid4
import json

import numpy as np

In [4]:
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances
from scipy.cluster.hierarchy import linkage, fcluster

# Configuration parameters
config = {
    'clustering': {
        'method': 'centroid',
        'min_cluster_size': 3,
        'threshold': 1.#0.7# 0.7045654963945799 #
    }
}



def centroid_linkage_distance(matrix):
    """Custom function to compute linkage with centroid method."""
    Z = linkage(matrix, method='centroid')
    return Z

def compute_centroids(embeddings, labels):
    """Compute centroids of clusters and return centroids in the order of labels."""
    unique_labels = np.unique(labels)
    centroids = np.zeros((len(unique_labels), embeddings.shape[1]))
    
    for i, label in enumerate(unique_labels):
        centroids[i] = embeddings[labels == label].mean(axis=0)
    
    return centroids, unique_labels

def map_labels_to_centroid_indices(labels, unique_labels):
    """Map labels to the indices of centroids."""
    label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
    centroid_indices = np.array([label_to_index[label] if label in unique_labels else -1 for label in labels])
    return centroid_indices

def cluster_embeddings(embeddings, config):
    # Compute pairwise distances
    distance_matrix = pairwise_distances(embeddings, metric='cosine')
    
    # Perform hierarchical clustering with centroid method
    Z = centroid_linkage_distance(distance_matrix)
    
    # Extract clusters based on distance threshold
    labels = fcluster(Z, t=config['clustering']['threshold'], criterion='distance')
    
    # Filter small clusters
    unique_labels, counts = np.unique(labels, return_counts=True)
    valid_labels = unique_labels[counts >= config['clustering']['min_cluster_size']]
    
    # Valid indices based on cluster size
    valid_indices = np.where(np.isin(labels, valid_labels))[0]
    
    # Compute centroids
    centroids, unique_labels = compute_centroids(embeddings, labels)
    
    # Map labels to the indices of centroids
    centroid_indices = map_labels_to_centroid_indices(labels, unique_labels)
    
    # Compute valid centroid indices (subset of centroid_indices)
    valid_centroid_indices = np.array([index for index, centroid in enumerate(centroids) if index in np.unique(centroid_indices[valid_indices])])

    return centroid_indices, centroids, valid_centroid_indices



In [5]:
def find_largest_span(points):
    # Use max to find the point with the largest span
    largest_span_point = max(points, key=lambda p: p.payload['span'])
    return largest_span_point

In [6]:
class SpeechDAL:
    def get_all_transcriptions(self) -> list:
        # Get all data
        raw_data = redis_client.lrange(f"transcriber", start=0, end=-1)
        return [json.loads(data) for data in raw_data]

    def get_all_diarizations(self) -> list:
        # Get all data
        raw_data = redis_client.lrange(f"diarizer", start=0, end=-1)
        return [json.loads(data) for data in raw_data]

    def lpush_diarizations(self,diarizations: str) -> None:
        for d in diarizations:
            redis_client.lpush(f"diarizer", json.dumps(d))

    def lpush_transcriptions(self, transcriptions) -> None:
        for t in transcriptions:
            redis_client.lpush(f"transcriber", json.dumps(t))

    def ltrim_transcriptions(self, meeting_id, slice: tuple):
        redis_client.ltrim(f"diarizer", int(slice[0]), int(slice[1]))

    def ltrim_diarizations(self, meeting_id, slice: tuple):
        redis_client.ltrim(f"diarizer", int(slice[0]), int(slice[1]))


In [7]:
@staticmethod
def prep_transcripts(transcriptions) -> pd.DataFrame:
    dfs = []
    for n, (t, seek) in enumerate(transcriptions):
        df = pd.DataFrame(t)[[2, 3, 4]]
        df.columns = ["start", "end", "speech"]
        df["start"] = pd.to_timedelta(df["start"], unit="s") + pd.Timestamp(seek)
        df["end"] = pd.to_timedelta(df["end"], unit="s") + pd.Timestamp(seek)
        df["trans_chunk"] = n
        dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)

@staticmethod
def prep_diarization(diarization) -> pd.DataFrame:
    d, embs,seek = diarization
    df = pd.DataFrame(d)
    df["start"] = pd.to_timedelta(df["start"], unit="s") + pd.Timestamp(seek)
    df["end"] = pd.to_timedelta(df["end"], unit="s") + pd.Timestamp(seek)

    return df,embs

In [8]:
def prep_diarizations(diarizations) -> pd.DataFrame:
    dfs = []
    embs = []
    for n, (d, emb, seek) in enumerate(diarizations):
        df = pd.DataFrame(d)
        df["start"] = pd.to_timedelta(df["start"], unit="s") + pd.Timestamp(seek)
        df["end"] = pd.to_timedelta(df["end"], unit="s") + pd.Timestamp(seek)
        df["diar_chunk"] = n
        dfs.append(df)
        embs.append(emb)
    return pd.concat(dfs).reset_index(drop=True),embs

In [9]:
from scipy.spatial.distance import squareform

In [10]:

embeddings_list = [] #redis
segments_df = None
speech_dal = SpeechDAL()
diarizations = speech_dal.get_all_diarizations()

In [11]:
segments_df,embs = prep_diarizations(diarizations)
embeddings_list.extend(embs)
embeddings = np.concatenate(embeddings_list, axis=0)
clusters,centroids,valid_clusters = cluster_embeddings(embeddings, config)
print(np.unique(clusters))

[0 1 2 3 4 5 6 7]


/tmp/ipykernel_2990372/2946564393.py:19: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  Z = linkage(matrix, method='centroid')


In [12]:
embs_mapping = segments_df[['diar_chunk','speaker_id']].sort_values(['diar_chunk','speaker_id']).drop_duplicates().set_index(['diar_chunk','speaker_id'])
embs_mapping['emb_id'] = np.arange(len(embs_mapping))
embs_mapping['cluster'] = clusters
segments_df = segments_df.set_index(['diar_chunk','speaker_id']).join(embs_mapping).reset_index().set_index('emb_id').drop(columns = ['speaker_id'])
        

In [13]:
segments_df['span'] = segments_df['end'] - segments_df['start']
segments_df['span'] = segments_df['span'].dt.seconds
speakers = segments_df[['cluster']].drop_duplicates().reset_index(drop=True)

cluster_span = segments_df.groupby('cluster')[['span']].sum()
cluster_span = cluster_span.to_dict()['span']
clusters = list(cluster_span.keys())

In [14]:
clusters

[0, 1, 2, 3, 4, 5, 6, 7]

In [15]:

new_points = []
for cluster in clusters:
    point_id = None

    centroid = centroids[cluster]
    search_results = client.search(collection_name="speakers", query_vector=list(centroid), limit=5,score_threshold=0.65)
    
    
    
    if len(search_results)==0:
        if cluster in valid_clusters:
            point_id = str(uuid4())
            new_points.append(PointStruct(id = point_id,vector = list(centroid),payload={"span":cluster_span[cluster]}))

    else:
        search_result = find_largest_span(search_results)
        point_id = search_result.id 
        remote_span = search_result.payload['span']
        

        span = cluster_span[cluster]
        if span > remote_span in valid_clusters:
            new_points.append(PointStruct(id = point_id,vector = list(centroid),payload={"span":span}))  
            
            
    segments_df.loc[segments_df['cluster']==cluster,'speaker'] = point_id
    
            
if len(new_points)>0:
    operation_info = client.upsert(
        collection_name="speakers",
        wait=True,
        points=new_points
    )       

segments_df = segments_df.sort_values('start').reset_index(drop=True)

    


/tmp/ipykernel_2990372/124942946.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '01d8d910-6c0d-436b-a021-b1cfb9e7c1ad' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  segments_df.loc[segments_df['cluster']==cluster,'speaker'] = point_id


In [16]:
diar_df = segments_df

In [17]:
def seconds_to_min_sec(seconds):
    minutes = int(seconds // 60)
    seconds = seconds % 60
    return f"{minutes}:{seconds:06.3f}"  # Format to show milliseconds
diar_df['start_'] = (diar_df['start']-diar_df['start'].min()).dt.seconds
diar_df['Start_min_sec'] = diar_df['start_'].apply(seconds_to_min_sec)

In [18]:
#diar_df = segments_df.groupby('speaker_switch').agg({'start':'first','end':'last','speaker':"first",'diar_chunk':'last'})

In [19]:
diar_df['span'] = (diar_df['end']-diar_df['start']).dt.seconds

In [20]:
speech_dal = SpeechDAL()
transcriptions = speech_dal.get_all_transcriptions()

In [21]:
trans_df = prep_transcripts(transcriptions).sort_values('start')

In [22]:

# Define a tolerance window (e.g., 2 seconds)
tolerance = pd.Timedelta(seconds=5)

# List to store results
results = []

# Iterate over each row in trans_df
for i, trans_row in trans_df.iterrows():
    max_intersection = pd.Timedelta(0)
    best_speaker = np.nan
    best_diar_chunk = np.nan
    
    # Compare with each row in diar_df
    for j, diar_row in diar_df.iterrows():
        # Calculate potential intersection
        intersection_start = max(diar_row['start'] - tolerance, trans_row['start'] - tolerance)
        intersection_end = min(diar_row['end'] + tolerance, trans_row['end'] + tolerance)
        intersection = intersection_end - intersection_start
        
        # Check if there is a positive intersection
        if intersection > pd.Timedelta(0):
            # If this is the largest intersection found, update speaker
            if intersection > max_intersection:
                max_intersection = intersection
                best_speaker = diar_row['speaker']
                best_diar_chunk = diar_row['diar_chunk']
    
    # Store the result with the best speaker match or nearest match
    result = trans_row.to_dict()
    result['speaker'] = best_speaker
    result['diar_chunk'] = best_diar_chunk
    result['intersection'] = max_intersection
    results.append(result)

# Convert results to DataFrame
merged_df = pd.DataFrame(results)

In [23]:
merged_df

,start,end,speech,trans_chunk,speaker,diar_chunk,intersection
0,2024-06-21 12:36:33.368864+00:00,2024-06-21 12:36:36.108864+00:00,Что у нас с тобой по спикерам? Пройдем все.,19,01d8d910-6c0d-436b-a021-b1cfb9e7c1ad,19.0,0 days 00:00:12.400084890
1,2024-06-21 12:36:37.248864+00:00,2024-06-21 12:36:39.268864+00:00,"По спикерам давай, по диарайзу.",19,2326f850-4157-4f2a-b826-e87318c1b942,19.0,0 days 00:00:11.281646859
2,2024-06-21 12:36:42.108864+00:00,2024-06-21 12:36:44.688864+00:00,Последний день перед отпуском на основной раб...,19,2326f850-4157-4f2a-b826-e87318c1b942,19.0,0 days 00:00:12.580000
3,2024-06-21 12:36:44.708864+00:00,2024-06-21 12:36:47.548864+00:00,"Блин, то по носу, то золотом. Все мое, блин.",19,2326f850-4157-4f2a-b826-e87318c1b942,19.0,0 days 00:00:12.040424448
4,2024-06-21 12:36:48.668864+00:00,2024-06-21 12:36:50.128864+00:00,У тебя два дня отпуск?,19,01d8d910-6c0d-436b-a021-b1cfb9e7c1ad,19.0,0 days 00:00:10.885280136
5,2024-06-21 12:36:51.408864+00:00,2024-06-21 12:36:53.148864+00:00,"Нет, у меня четыре со вторника.",19,01d8d910-6c0d-436b-a021-b1cfb9e7c1ad,19.0,0 days 00:00:11.494414261
6,2024-06-21 12:36:53.258864+00:00,2024-06-21 12:36:55.498864+00:00,"по пятницу на основной работе,",18,2326f850-4157-4f2a-b826-e87318c1b942,18.0,0 days 00:00:12.231511036
7,2024-06-21 12:36:55.518864+00:00,2024-06-21 12:36:58.058864+00:00,а здесь я отдохну,18,2326f850-4157-4f2a-b826-e87318c1b942,18.0,0 days 00:00:12.540000
8,2024-06-21 12:36:58.058864+00:00,2024-06-21 12:36:59.758864+00:00,"в среду, в четверг, завтра я в строю буду.",18,2326f850-4157-4f2a-b826-e87318c1b942,18.0,0 days 00:00:11.700000
9,2024-06-21 12:37:00.658864+00:00,2024-06-21 12:37:01.218864+00:00,Ага.,18,01d8d910-6c0d-436b-a021-b1cfb9e7c1ad,18.0,0 days 00:00:10.362376910


In [48]:
diar_df

,diar_chunk,start,end,cluster,span,speaker,start_,Start_min_sec
0,19,2024-06-21 12:36:33.267352964+00:00,2024-06-21 12:36:36.170578771+00:00,0,2,dac38808-0f6d-42c5-a0fc-b355e453db00,0,0:00.000
1,19,2024-06-21 12:36:36.730850418+00:00,2024-06-21 12:36:40.720663660+00:00,0,3,dac38808-0f6d-42c5-a0fc-b355e453db00,3,0:03.000
2,19,2024-06-21 12:36:40.907420876+00:00,2024-06-21 12:36:41.348847022+00:00,6,0,None,7,0:07.000
3,19,2024-06-21 12:36:41.179067735+00:00,2024-06-21 12:36:42.554279959+00:00,0,1,dac38808-0f6d-42c5-a0fc-b355e453db00,7,0:07.000
4,19,2024-06-21 12:36:42.876860604+00:00,2024-06-21 12:36:53.352242608+00:00,0,10,dac38808-0f6d-42c5-a0fc-b355e453db00,9,0:09.000
...,...,...,...,...,...,...,...,...
106,0,2024-06-21 13:39:55.050035477+00:00,2024-06-21 13:39:56.798762132+00:00,0,1,dac38808-0f6d-42c5-a0fc-b355e453db00,3801,63:21.000
107,0,2024-06-21 13:39:57.223210350+00:00,2024-06-21 13:40:09.022870791+00:00,0,11,dac38808-0f6d-42c5-a0fc-b355e453db00,3803,63:23.000
108,0,2024-06-21 13:40:09.294517650+00:00,2024-06-21 13:40:10.550884374+00:00,0,1,dac38808-0f6d-42c5-a0fc-b355e453db00,3816,63:36.000
109,0,2024-06-21 13:40:11.841206954+00:00,2024-06-21 13:40:12.316588958+00:00,0,0,dac38808-0f6d-42c5-a0fc-b355e453db00,3818,63:38.000


In [39]:
segments = trans_df.to_dict("records")

In [24]:
segments = trans_df.to_dict("records")

if len(segments) > 0:
    for seg in segments:
        diar_df["intersection"] = np.minimum(diar_df["end"], seg["end"]) - np.maximum(
            diar_df["start"], seg["start"]
        )
        speaker_ = diar_df[
            (diar_df["intersection"] == diar_df["intersection"].max())
            & (diar_df["intersection"] > pd.Timedelta(0))
        ][["speaker", "diar_chunk"]]
        if len(speaker_) > 0:
            seg["speaker"] = speaker_["speaker"].iloc[0]


        # ToDo: Dima
        else:
            seg["speaker"] = np.nan

In [29]:
diar_df.head(2)

,diar_chunk,start,end,cluster,span,speaker,start_,Start_min_sec,intersection
0,19,2024-06-21 12:36:33.267352964+00:00,2024-06-21 12:36:36.170578771+00:00,0,2,dac38808-0f6d-42c5-a0fc-b355e453db00,0,0:00.000,-1 days +23:53:29.851714771
1,19,2024-06-21 12:36:36.730850418+00:00,2024-06-21 12:36:40.720663660+00:00,0,3,dac38808-0f6d-42c5-a0fc-b355e453db00,3,0:03.000,-1 days +23:53:34.401799660


In [30]:
trans_df.head(2)

,start,end,speech,trans_chunk
52,2024-06-21 12:36:33.258864+00:00,2024-06-21 12:36:43.238864+00:00,видимо уже кто-то для них легенем и они как это все видят они стартап и соответственно стартап,19
53,2024-06-21 12:36:43.238864+00:00,2024-06-21 12:36:48.058864+00:00,в котором видимо не очень много денег если бы их было много им было бы не так плохо короче на,19


In [25]:
pd.options.display.max_colwidth = None

In [26]:
pd.DataFrame(segments)

,start,end,speech,trans_chunk,speaker
0,2024-06-21 12:36:33.258864+00:00,2024-06-21 12:36:43.238864+00:00,видимо уже кто-то для них легенем и они как это все видят они стартап и соответственно стартап,19,dac38808-0f6d-42c5-a0fc-b355e453db00
1,2024-06-21 12:36:43.238864+00:00,2024-06-21 12:36:48.058864+00:00,в котором видимо не очень много денег если бы их было много им было бы не так плохо короче на,19,dac38808-0f6d-42c5-a0fc-b355e453db00
2,2024-06-21 12:36:48.058864+00:00,2024-06-21 12:36:53.218864+00:00,то как они выстраивают бизнес и видимо как я понял они хотят сделать так как бы мы там,19,dac38808-0f6d-42c5-a0fc-b355e453db00
3,2024-06-21 12:36:53.258864+00:00,2024-06-21 12:36:57.198864+00:00,"как-то квалифицируем лида, понимаем, что у него есть потребность, есть нужда, есть, там,",18,dac38808-0f6d-42c5-a0fc-b355e453db00
4,2024-06-21 12:36:57.218864+00:00,2024-06-21 12:37:01.418864+00:00,"экшендайзер, у них нет салюшна, как бы, и им интересно, и дальше их передаем",18,dac38808-0f6d-42c5-a0fc-b355e453db00
5,2024-06-21 12:37:01.418864+00:00,2024-06-21 12:37:04.998864+00:00,"уже на практически клоузе, то есть какое-то делаем при дискавери.",18,dac38808-0f6d-42c5-a0fc-b355e453db00
6,2024-06-21 12:37:05.238864+00:00,2024-06-21 12:37:08.938864+00:00,"Я говорю, ну, вы, ебал, долбаребы, типа, ну, как бы, за 1600 долларов, как бы,",18,dac38808-0f6d-42c5-a0fc-b355e453db00
7,2024-06-21 12:37:08.998864+00:00,2024-06-21 12:37:13.278864+00:00,"что вам делать при дискавери, да, как бы. Я говорю, мы можем вам за счет",18,dac38808-0f6d-42c5-a0fc-b355e453db00
8,2024-06-21 12:37:13.438864+00:00,2024-06-21 12:37:33.078864+00:00,"пару вопросов задать типа перед встречей, там типа знаешь, как мы делали, couple questions before the meeting if you don't mind, и у них задача такая, они хотят максимально быстро закрыть какое-то количество deals и как бы нас использовать как вот таких квалифаеров и",17,NaN
9,2024-06-21 12:37:33.258864+00:00,2024-06-21 12:37:48.558864+00:00,"типа сорсеров, лидов, я им сказал, как ты помнишь, на звонке, я говорю, блядь, это все заебись, но мы вообще интерпрайз, ребята, мы вообще, блядь, настроены на большой чек нахуй, и нам, как бы, нужен, как бы, ROI такой довольно long term, и он выше намного, поэтому и мы чарлить хотим у вас больше денег в будущем.",16,dac38808-0f6d-42c5-a0fc-b355e453db00


In [27]:
diar_df.head(50)


,diar_chunk,start,end,cluster,span,speaker,start_,Start_min_sec,intersection
0,19,2024-06-21 12:36:33.267352964+00:00,2024-06-21 12:36:36.170578771+00:00,0,2,dac38808-0f6d-42c5-a0fc-b355e453db00,0,0:00.000,-1 days +23:53:29.851714771
1,19,2024-06-21 12:36:36.730850418+00:00,2024-06-21 12:36:40.720663660+00:00,0,3,dac38808-0f6d-42c5-a0fc-b355e453db00,3,0:03.000,-1 days +23:53:34.401799660
2,19,2024-06-21 12:36:40.907420876+00:00,2024-06-21 12:36:41.348847022+00:00,6,0,None,7,0:07.000,-1 days +23:53:35.029983022
3,19,2024-06-21 12:36:41.179067735+00:00,2024-06-21 12:36:42.554279959+00:00,0,1,dac38808-0f6d-42c5-a0fc-b355e453db00,7,0:07.000,-1 days +23:53:36.235415959
4,19,2024-06-21 12:36:42.876860604+00:00,2024-06-21 12:36:53.352242608+00:00,0,10,dac38808-0f6d-42c5-a0fc-b355e453db00,9,0:09.000,-1 days +23:53:47.033378608
5,18,2024-06-21 12:36:53.267352964+00:00,2024-06-21 12:37:03.182463321+00:00,0,9,dac38808-0f6d-42c5-a0fc-b355e453db00,20,0:20.000,-1 days +23:53:56.863599321
6,18,2024-06-21 12:37:03.386198465+00:00,2024-06-21 12:37:05.100969263+00:00,0,1,dac38808-0f6d-42c5-a0fc-b355e453db00,30,0:30.000,-1 days +23:53:58.782105263
7,18,2024-06-21 12:37:05.457505766+00:00,2024-06-21 12:37:13.301308822+00:00,0,7,dac38808-0f6d-42c5-a0fc-b355e453db00,32,0:32.000,-1 days +23:54:06.982444822
8,18,2024-06-21 12:37:12.265655171+00:00,2024-06-21 12:37:12.316588958+00:00,2,0,0e0b4403-963b-45cb-8605-07cae5a5ad70,38,0:38.000,-1 days +23:54:05.997724958
9,17,2024-06-21 12:37:33.776690825+00:00,2024-06-21 12:37:35.847998126+00:00,0,2,dac38808-0f6d-42c5-a0fc-b355e453db00,60,1:00.000,-1 days +23:54:29.529134126


In [30]:
rename_dict = {
    
    'a1e24d98-918b-4633-8a4e-a4613949c0bc':'Ryabenko',
    '1c6cac3c-0eba-4404-a6fc-ab1f54992292':'Ikenna',
    'e92fd67c-d305-4bd4-a1d4-81dc86e2f063':'Me',
    'f39f85d8-2c76-4ee5-bb70-dc18f7c39af5':'Me',
    '7c0d4578-e6e2-4216-b3fa-bc9fa31162c9':'Me',
    '4384a580-e3d3-4b96-bd01-5412318bdbda':'ANdrew',
    'd22658aa-ab7a-4aea-a322-ff86c2274b18':'Olya',
    'feed0fdf-8332-4c6c-842d-b8fcc1543508':'Liza',
    '77270ffb-54a5-4a23-8b10-d273347a7233':'Egor',
    'e419df06-381b-4de1-a8aa-dca078326af8':'ANton',
    '1ccf5e92-978f-4fd6-861e-d21afcc4294c':'liza',
    '1cbc5a49-3521-4f67-8fbc-9601c50a6753':'David',
    '926bad2e-a55e-498e-9cef-68344f491d88':'Me',
    'be6ac578-3d51-435a-9d70-5f99f4e49c79':'Katz',
    '4cad8508-a340-4fdb-923b-0db5c632ebb2':'Dima Blaster',
    '5661486b-45e2-40d6-addb-17d59ad9832b':'Blasteritsa'


    

               
               }

segments_df['point_id'] = segments_df['point_id'].replace(rename_dict)
segments_df = segments_df.sort_values('start').reset_index(drop=True)
segments_df.tail(50)

KeyError: 'point_id'

In [ ]:
segments_df

,emb_id,start,end,cluster,span,point_id
0,37,2024-06-20 18:59:01.039337964+00:00,2024-06-20 18:59:02.618285333+00:00,0,1,liza
1,36,2024-06-20 18:59:01.039337964+00:00,2024-06-20 18:59:04.791460205+00:00,0,3,liza
2,36,2024-06-20 18:59:05.029151207+00:00,2024-06-20 18:59:10.343242888+00:00,0,5,liza
3,38,2024-06-20 18:59:08.170068015+00:00,2024-06-20 18:59:08.204023873+00:00,2,0,None
4,37,2024-06-20 18:59:08.204023873+00:00,2024-06-20 18:59:08.577538304+00:00,0,0,liza
...,...,...,...,...,...,...
61,4,2024-06-20 19:24:37.270237795+00:00,2024-06-20 19:24:41.022360036+00:00,6,3,7a9b58aa-d154-40fb-a041-ae5ddcc77521
62,3,2024-06-20 19:27:31.039337964+00:00,2024-06-20 19:27:41.022360036+00:00,6,9,7a9b58aa-d154-40fb-a041-ae5ddcc77521
63,2,2024-06-20 19:27:40.122529815+00:00,2024-06-20 19:27:40.920492463+00:00,12,0,None
64,0,2024-06-20 19:30:41.039337964+00:00,2024-06-20 19:30:50.954448321+00:00,6,9,7a9b58aa-d154-40fb-a041-ae5ddcc77521
